# Soft and Hard Voting

Voting Classifier

A Voting Classifier is a machine learning model that trains on an ensemble of numerous models and predicts an output (class) 
based on their highest probability of chosen class as the output.If we have trained a few classifiers, each one achieving about 
80% accuracy.You may have a Logistic Regression classifier, an SVM classifier, a Random Forest classifier, a K-Nearest Neighbors
classifier, and perhaps a few more . A very simple way to create an even better classifier is to aggregate the predictions of 
each classifier and predict the class that gets the most votes. This majority-vote classifier is called a hard voting classifier.
Somewhat surprisingly, this voting classifier often achieves a higher accuracy than the best classifier in the ensemble. 
In fact, even if each classifier is a weak learner (meaning it does only slightly better than random guessing), the ensemble 
can still be a strong learner (achieving high accuracy), provided there are a sufficient number of weak learners and they are 
sufficiently diverse. Voting Classifier supports two types of votings:
    
Hard Voting: In hard voting, the predicted output class is a class with the highest majority of votes i.e the class which had 
    the highest probability of being predicted by each of the classifiers. Suppose three classifiers predicted the output 
    class(A, A, B), so here the majority predicted A as output. Hence A will be the final prediction.

Soft Voting: In soft voting, the output class is the prediction based on the average of probability given to that class. 
    Suppose given some input to three models, the prediction probability for class A = (0.30, 0.47, 0.53) and 
    B = (0.20, 0.32, 0.40). So the average for class A is 0.4333 and B is 0.3067, the winner is clearly class 
    A because it had the highest probability averaged by each classifier.

Note: Make sure to include a variety of models to feed a Voting Classifier to be sure that the error made by one might be 
      resolved by the other.
